# Fixture Observation

#### This script creates a fixture observation for each match, combining the home and away teams statistics, as well as the prediction target - pre-match odds

In [1]:
import boto3
import pandas as pd
import numpy as np
import json
from io import StringIO


s3 = boto3.resource('s3')
bucket_name = 'betfairex'


def read_csv_from_s3(bucket_name, object_key):
    response = s3.Object(bucket_name, object_key).get()

    s3_contents = response['Body'].read()
    df = pd.read_csv(StringIO(s3_contents.decode('utf-8')))

    return df

In [2]:
bucket = s3.Bucket(bucket_name)

In [3]:
all_keys = [obj.key for obj in bucket.objects.all()]

# keys are in the regular expression as follows:
# YYYY-MM-DD_[HOME_TEAM] v [AWAY_TEAM].csv

In [4]:
LEAGUE_INFO = pd.read_csv('league_info.csv')
TEAM_INFO = pd.read_csv('team_info.csv')

def get_match_info(object_key):
    match_name = object_key[:-4]
    match_date_str = match_name.split('_')[0]
    fixture = match_name.split('_')[1]
    home_team = fixture.split(' v ')[0]
    away_team = fixture.split(' v ')[1]
    
    match_info = {
        'match_date': match_date_str,
        'home_team': home_team,
        'away_team': away_team,
    }    
    return match_info

def get_betfair_exchange_prematch_odds(bucket_name, object_key):
    match_info = get_match_info(object_key)
    
    odds_df = read_csv_from_s3(bucket_name, object_key)
    
    odds_df = odds_df.sort_values("timestamp", ascending=True)
    prematch_df = odds_df[~odds_df['inplay']]
    
    home_team_df = prematch_df[prematch_df['selection'] == match_info['home_team']]
    away_team_df = prematch_df[prematch_df['selection'] == match_info['away_team']]
    draw_df = prematch_df[prematch_df['selection'] == "Draw"]
    
    home_prematch_odds = home_team_df.tail(1)['odds'].iloc[0]
    away_prematch_odds = away_team_df.tail(1)['odds'].iloc[0]
    draw_prematch_odds = draw_df.tail(1)['odds'].iloc[0]
    
    prematch_odds = {
        'match_date': match_info['match_date'],
        match_info['home_team']: home_prematch_odds,
        match_info['away_team']: away_prematch_odds,
        'Draw': draw_prematch_odds
    }
    return prematch_odds    
    

In [6]:
TEAM_MAPPING_BETFAIR_TO_APIFOOTBALL = {
    "Athletic Bilbao": "Athletic Club",
    "Betis": "Real Betis"
}

In [10]:
# Configuration constants

SERVER = "v3.football.api-sports.io"
API_KEY = "1f5008c4f33481203631d90c7a81c5e5"

from pandas.tseries.offsets import Day
import http.client
import datetime as dt

SERVER = "v3.football.api-sports.io"



conn = http.client.HTTPSConnection(SERVER)

headers = {
  'x-rapidapi-host': SERVER,
  'x-rapidapi-key': API_KEY
}


def get_team_stats(season, league_id, team_id, as_of_date):
    # as_of_date in format YYYY-MM-DD
    conn.request("GET", "/teams/statistics?season=%s&league=%s&team=%s&date=%s" % (season, league_id, team_id, as_of_date), headers=headers)
    res = conn.getresponse()
    data = res.read()

    team_stats_raw = data.decode("utf-8")
    team_stats = json.loads(team_stats_raw)["response"]
    
    return team_stats
    
def get_team_stats_for_fixture(object_key):
    match_info = get_match_info(object_key)
    
    match_date = match_info['match_date']
    match_date = dt.datetime.strptime(match_date, "%Y-%m-%d")
    as_of_date = match_date - Day(1)
    as_of_date = as_of_date.strftime("%Y-%m-%d")

    match_year = match_date.year
    match_month = match_date.month
    
    if match_month <= 6:
        match_year -= 1

    
    home_team = TEAM_MAPPING_BETFAIR_TO_APIFOOTBALL.get(match_info['home_team'], match_info['home_team'])
    home_team_info = TEAM_INFO.query("name == @home_team").to_dict('records')[0]
    home_team_id = home_team_info['team_id']
    home_team_league_id = home_team_info['league_id']
    home_team_stats = get_team_stats(match_year, home_team_league_id, home_team_id, as_of_date)
    
    
    away_team = TEAM_MAPPING_BETFAIR_TO_APIFOOTBALL.get(match_info['away_team'], match_info['away_team'])
    away_team_info = TEAM_INFO.query("name == @away_team").to_dict('records')[0]
    away_team_id = away_team_info['team_id']
    away_team_league_id = away_team_info['league_id']
    away_team_stats = get_team_stats(match_year, away_team_league_id, away_team_id, as_of_date)
    
    return home_team_stats, away_team_stats
    

In [11]:
def calc_form_score(form):
    form = form[-6:]
    form_dict = {
        'W': 1,
        'D': 0,
        'L': -1
    }
    
    form_score = 0
    for res in form[:3]:
        form_score += 0.5 * form_dict[res]
    for res in form[3:]:
        form_score += 1 * form_dict[res]
    return form_score

def generate_features(object_key):
    row = {}
    match_info = get_match_info(object_key)
    
    # Index
    row['match_date'] = match_info['match_date']
    row['home_team'] = match_info['home_team']
    row['away_team'] = match_info['away_team']
    
    # Features
    home_team_stats, away_team_stats = get_team_stats_for_fixture(object_key)
    
    row['home_team_avg_home_goals'] = home_team_stats['goals']['for']['average']['home']
    row['home_team_avg_total_goals'] = home_team_stats['goals']['for']['average']['total']
    row['away_team_avg_away_goals'] = away_team_stats['goals']['for']['average']['away']
    row['away_team_avg_total_goals'] = away_team_stats['goals']['for']['average']['total']
    
    row['home_team_form_score'] = calc_form_score(home_team_stats['form'])
    row['away_team_form_score'] = calc_form_score(away_team_stats['form'])
    
    # Labels
    prematch_odds = get_betfair_exchange_prematch_odds(bucket_name, object_key)
    row['odds_home_team'] = prematch_odds[match_info['home_team']]
    row['odds_away_team'] = prematch_odds[match_info['away_team']]
    row['odds_draw'] = prematch_odds['Draw']
    
    return row

In [15]:
from tqdm import tqdm
import time
rows = []


for i in tqdm(range(1, len(all_keys) + 1)):
    if i % 50 == 0:
        time.sleep(10)
    
    key = all_keys[-i]
    
    match_info = get_match_info(key)
    if match_info['match_date'][5:7] == '08':
        continue
    
    try:
        row = generate_features(key)
        rows.append(row)
    except:
        print("Failed to generate feature for %s." % key)

 40%|████████████████▎                        | 187/469 [00:56<00:51,  5.51it/s]

Failed to generate feature for 2024-05-19_Valencia v Girona.csv.
Failed to generate feature for 2024-05-19_Betis v Real Sociedad.csv.
Failed to generate feature for 2024-05-19_Barcelona v Rayo Vallecano.csv.
Failed to generate feature for 2024-05-19_Atletico Madrid v Osasuna.csv.


 41%|████████████████▋                        | 191/469 [00:56<00:38,  7.22it/s]

Failed to generate feature for 2024-05-19_Athletic Bilbao v Sevilla.csv.
Failed to generate feature for 2024-05-19_Alaves v Getafe.csv.
Failed to generate feature for 2024-05-16_Real Sociedad v Valencia.csv.
Failed to generate feature for 2024-05-16_Las Palmas v Betis.csv.


 42%|█████████████████                        | 195/469 [00:56<00:30,  9.03it/s]

...

 80%|████████████████████████████████▊        | 375/469 [02:14<00:14,  6.44it/s]

Failed to generate feature for 2023-11-17_Valladolid v Leganes.csv.


 86%|███████████████████████████████████▎     | 404/469 [02:30<01:03,  1.03it/s]

Failed to generate feature for 2023-10-20_Espanyol v Leganes.csv.
Failed to generate feature for 2023-10-14_Espanyol v Valladolid.csv.


100%|█████████████████████████████████████████| 469/469 [02:49<00:00,  2.77it/s]


In [16]:
df = pd.DataFrame.from_records(rows)

In [17]:
df

,match_date,home_team,away_team,home_team_avg_home_goals,home_team_avg_total_goals,away_team_avg_away_goals,away_team_avg_total_goals,home_team_form_score,away_team_form_score,odds_home_team,odds_away_team,odds_draw
0,2024-12-20,Girona,Valladolid,1.9,1.4,0.8,0.7,-0.5,-2.0,1.38,11.5,5.30
1,2024-12-18,Villarreal,Rayo Vallecano,2.1,1.8,0.8,1.1,-1.0,-0.5,1.76,5.6,3.90
2,2024-12-18,Espanyol,Valencia,1.4,0.9,0.4,0.9,-1.5,-3.0,2.68,3.5,2.94
3,2024-12-15,Villarreal,Betis,2.3,1.8,0.8,1.1,0.0,-1.5,1.95,4.3,3.95
4,2024-12-15,Real Sociedad,Las Palmas,1.1,1.0,1.4,1.4,1.5,1.5,1.36,12.5,5.50
...,...,...,...,...,...,...,...,...,...,...,...,...
385,2023-09-03,Atletico Madrid,Sevilla,3.0,3.3,3.0,1.7,1.0,-1.5,1.59,7.0,4.60
386,2023-09-02,Real Madrid,Getafe,0.0,2.0,0.0,0.3,1.5,0.0,1.38,12.0,5.30
387,2023-09-02,Girona,Las Palmas,3.0,2.0,0.0,0.3,1.0,-0.5,1.68,6.2,4.00
388,2023-09-02,Betis,Rayo Vallecano,0.0,1.3,2.0,1.3,0.0,0.5,2.10,4.2,3.55


In [18]:
df.to_csv('basic_dataset.csv')